# Evaluating combo SAEs

In [1]:
from main import *

/Users/charlesoneill/miniconda3/envs/anu/lib/python3.12/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/charlesoneill/miniconda3/envs/anu/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/charlesoneill/miniconda3/envs/anu/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## Max activating features

## Linearity of SAE sizes

In [1]:
# Train a larger regular SAE, hidden size 32 * 768 = 24576
import torch
from main_regular import main_regular_sae  # Adjust the import based on the file location

# Manually specify the arguments
layer = 9
model_type = 'gated'
n_batches = 5000
l1_coefficient = 3e-4
batch_size = 4
lr = 0.001
projection_up = 32
repo_name = "regular-sae"

# Call the function with specified arguments
main_regular_sae(layer, model_type, n_batches, l1_coefficient, batch_size, lr, projection_up, repo_name)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Running on cpu...


wandb: Currently logged in as: charlieoneill. Use `wandb login --relogin` to force relogin


/Users/charlesoneill/miniconda3/envs/anu/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


...

Batch 75 Test Loss 0.0719 | Reconstruction Error 0.0368 | L0 Loss 70.5000 | Dead Neurons: 89.80%


Training Batches:   2%|▏         | 100/5000 [01:08<2:19:00,  1.70s/it]

Batch 100 Test Loss 0.0882 | Reconstruction Error 0.0451 | L0 Loss 77.2496 | Dead Neurons: 93.99%


Training Batches:   2%|▏         | 115/5000 [01:17<54:36,  1.49it/s]  


KeyboardInterrupt: 

In [1]:
# Train a smaller regular SAE, hidden size 16_000

In [4]:
# Train a smaller error SAE, hidden size 16_000
layer = 9
model_type = 'gated'
n_epochs = 100
l1_coefficient = 3e-4
batch_size = 2048
lr = 0.001
projection_up = 16
repo_name = "error-saes"

error_sae = main(layer, model_type, n_epochs, l1_coefficient, projection_up, batch_size, lr, repo_name, return_model=True, save_model=False)

Running on cpu...
Loading existing Z activations for layer 9...
Loading existing SAE errors and original Z activations for layer 9...
Length of dataset = 10000
Shape of x = torch.Size([2048, 768])
Shape of y = torch.Size([2048, 768])


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  3.59it/s]


Initial Test Loss 0.1996 | Initial Reconstruction Error 0.0334 | Initial L0 Loss 6154.4863 | Initial Dead Neurons: 0.00%


Training Epochs:   1%|          | 1/100 [00:01<02:42,  1.64s/it]

Epoch 1 Test Loss 0.0584 | Reconstruction Error 0.0286 | L0 Loss 88.4235 | Dead Neurons: 3.14%


...

Epoch 98 Test Loss 0.0401 | Reconstruction Error 0.0183 | L0 Loss 64.9815 | Dead Neurons: 95.42%


Training Epochs:  99%|█████████▉| 99/100 [02:35<00:01,  1.61s/it]

Epoch 99 Test Loss 0.0401 | Reconstruction Error 0.0183 | L0 Loss 64.3420 | Dead Neurons: 95.43%


Training Epochs: 100%|██████████| 100/100 [02:37<00:00,  1.57s/it]


Epoch 100 Test Loss 0.0400 | Reconstruction Error 0.0182 | L0 Loss 65.5345 | Dead Neurons: 95.43%
Final Reconstruction Error: 0.0182
Final L0 Loss: 65.5345
